In [1]:
import pandas as pd
import numpy as np
import import_ipynb
from datetime import datetime

In [2]:
data = pd.read_csv("C:\\Users\\Sampreeth Salveru\\Downloads\\AG_Discrete_DS06-Jul-2020.csv")
excel = pd.read_excel("C:\\Users\\Sampreeth Salveru\\Downloads\\riskQA.xlsx",index_col="Quality Attributes")
filter_col = [col for col in data if col.startswith(('Drug','Load'))]


data['Date'] = pd.to_datetime(data['PSDate'])
data = data.set_index("Date")
data = data.last('3M')

In [3]:
filter_excel = list(excel.index)

In [4]:
# WECO Rule 1
def weco1(col, ucl, lcl):
    if np.isnan(ucl) == False and np.isnan(lcl) == False:
        y = np.array(data[col].dropna())
        rule1 = np.array([])
        for i in range(0,len(y)):
            if(y[i] > ucl or y[i] < lcl):
                rule1 = np.append(rule1, i)
        rule1 = rule1.astype(np.int)
        return rule1
    elif np.isnan(ucl) == True and np.isnan(lcl) == False:
        y = np.array(data[col].dropna())
        rule1 = np.array([])
        for i in range(0,len(y)):
            if(y[i] < lcl):
                rule1 = np.append(rule1, i)
        rule1 = rule1.astype(np.int)
        return rule1
    elif np.isnan(ucl) == False and np.isnan(lcl) == True:
        y = np.array(data[col].dropna())
        rule1 = np.array([])
        for i in range(0,len(y)):
            if(y[i] > ucl):
                rule1 = np.append(rule1, i)
        rule1 = rule1.astype(np.int)
        return rule1
    else: 
        return []

# WECO Rule 4
def weco4(col, mean):
    if np.isnan(mean) == True:
        return []
    else:
        x = data[col].dropna()
        anomalies = x - mean
        signs = np.sign(anomalies)
        signs = np.array(signs)
        A = signs.cumsum()
        A[8:] -= A[:-8]
        rule4 = (np.abs(A)==8).nonzero()[0]
        return rule4

# WECO Rule 5
def weco5(col):
    x = data[col].dropna()
    nub = np.sign(np.diff(x)).astype(np.int8)
    N = 6
    C = nub.cumsum()
    C[N:] -= C[:-N]
    rule5 =(np.abs(C)==N).nonzero()[0]
    return rule5

In [5]:
df = pd.DataFrame(columns = ["Low Risk Signal","Medium Risk Signal","High Risk Signal"])

In [6]:
def findrisk(col):
    usl = excel.loc[col,"USL"]
    lsl = excel.loc[col,"LSL"]
    ucl = excel.loc[col,"UCL"]
    lcl = excel.loc[col,"LCL"]
    mean = excel.loc[col,"Mean"]
    std = excel.loc[col,"Sigma"]
    ind = weco1(col,ucl, lcl)
    
    low = 0; med = 0; high = 0
    low1 = 0
    med1 = 0
    high1 = 0
    

    cond_1 = lcl-std
    cond_2 = lcl-lsl
    cond_3 = 3*std
    cond_1_1 = ucl+std
    cond_2_1 = usl-ucl
    
#     y = np.array(data[col][ind])
    y = np.array(data[col].dropna())[ind]
    if len(y) != 0:
        for i in range(0,len(y)):
            if y[i] < mean:
                if y[i]<lsl:
                    high = high + 1
                elif (y[i] < cond_1 and cond_2>=cond_3) or (y[i] < lcl and cond_2 < cond_3):
                    med = med+1
                elif y[i] >= cond_1 and cond_2>=cond_3:
                    low = low + 1
    #             else:
    #                 return
            if y[i] > mean:
                if y[i]>usl:
                    high1 = high1 + 1
                elif (y[i] > cond_1_1 and cond_2_1>=cond_3) or (y[i] > ucl and cond_2_1 < cond_3):
                    med1 = med1 + 1
                elif y[i] <= cond_1_1 and cond_2_1>=cond_3:
                    low1 = low1 + 1
    #             else:
    #                 return
        return np.array([low+low1,med+med1,high+high1])
    else:
        return np.array([low+low1,med+med1,high+high1])


In [7]:
def findrisk4(col):
    usl = excel.loc[col,"USL"]
    lsl = excel.loc[col,"LSL"]
    ucl = excel.loc[col,"UCL"]
    lcl = excel.loc[col,"LCL"]
    mean = excel.loc[col,"Mean"]
    std = excel.loc[col,"Sigma"]
    ind = weco4(col,mean)
    
        
    low = 0; med = 0; high = 0
    low1 = 0; med1 = 0; high1 = 0
    
    y = np.array(data[col].dropna())[ind]
#     y = y[~np.isnan(y)]

    if len(y) != 0:
        ymean = np.nanmean(y)
        cond_4 = mean-ymean
        for i in range(0,len(y)):
            if y[i] < mean:
                if y[i] < lsl:
                    high = high + 1
                elif cond_4 > std and y[i] > lsl:
                    med = med + 1
                elif cond_4 <= std and y[i] > lsl:
                    low = low + 1
#                 else:
#                     return
            if y[i] > mean:
                if y[i] > usl:
                    high1 = high1 + 1
                elif cond_4 > std and y[i] > usl:
                    med1 = med1 + 1
                elif cond_4 <= std and y[i] > usl:
                    low1 = low1 + 1
#                 else:
#                     return
        return np.array([low+low1,med+med1,high+high1])
    else:
        return np.array([low,med,high])
            

In [8]:
def findrisk5(col):
    usl = excel.loc[col,"USL"]
    lsl = excel.loc[col,"LSL"]
    mean = excel.loc[col,"Mean"]
    std = excel.loc[col,"Sigma"]
    ind = weco5(col)
    
    low = 0; med = 0; high = 0
    low1 = 0; med1 = 0; high1 = 0
    
    y = np.array(data[col].dropna())[ind]
    if len(y) != 0:
        cond_5 = mean-(2*std)
        cond_6 = mean+(2*std)
        for i in range(0,len(y)):
            if y[i] < lsl or y[i] > usl:
                high = high + 1
            elif y[i] > cond_6 or y[i] < cond_5:
                med = med + 1
            elif cond_5<y[i]<cond_6:
                low = low + 1
        return np.array([low, med, high])
    else:
        return np.array([low,med,high])

In [9]:
def total(col):
    total = findrisk(col)+findrisk4(col)+findrisk5(col)
    return total


In [10]:
for x in range(len(filter_excel)):
    df.loc[filter_col[x]] = total(filter_col[x])

In [11]:
df["Date Range Beg"] = data.index[0].date()
df["Date Range End"] = data.index[-1].date()
df

,Low Risk Signal,Medium Risk Signal,High Risk Signal,Date Range Beg,Date Range End
DrugSubstance_Calcium,0,12,0,2019-10-18,2019-12-21
DrugSubstance_Copper,0,0,0,2019-10-18,2019-12-21
DrugSubstance_DNA,0,0,0,2019-10-18,2019-12-21
DrugSubstance_Glycine,0,0,0,2019-10-18,2019-12-21
DrugSubstance_HCP,0,0,0,2019-10-18,2019-12-21
DrugSubstance_Histidine,6,0,0,2019-10-18,2019-12-21
DrugSubstance_MicrobialLoad,0,0,0,2019-10-18,2019-12-21
DrugSubstance_MouseIgG,0,0,0,2019-10-18,2019-12-21
DrugSubstance_Oligo_AsialylN_glycans,0,0,0,2019-10-18,2019-12-21
DrugSubstance_Oligo_di_sialylN_glycans,1,16,0,2019-10-18,2019-12-21
